# HW05 Subword_tokenization

## 1. Делаем упрощенную версию byte pair encoding

In [1]:
import re
import numpy as np
import pandas as pd
from scipy.sparse import lil_matrix
from collections import Counter, defaultdict
from tokenizers import CharBPETokenizer, Tokenizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score

In [2]:
with open('data/lenta.txt', 'r', encoding='utf-8') as f: text = f.read()

In [3]:
text = text[:50000].lower()

In [4]:
def statistics(text):
    frequencies = {}
    for i in range(len(text)-1):
        next_symbol = text[i+1]
        pair = text[i] + next_symbol
        if pair in frequencies:
            frequencies[pair] += 1
        else:
            frequencies[pair] = 1
    return frequencies

In [5]:
def merge_most_freq(text, dictionary, k):
    sort_dict = {}
    keys = list(dictionary.keys())
    keys = sorted(keys, key=lambda x: dictionary[x], reverse=True)
    keys = keys[:k]
    for key in keys:
        sort_dict[key] = dictionary[key]
    result = ()
    process = True
    for ind, sym in enumerate(text):
        if process:
            if ind < len(text)-1:
                next_symbol = text[ind+1]
            else:
                result += tuple(sym, )
                break
            pair = sym + next_symbol
            if pair in sort_dict: 
                result += (pair, )
                process = False
            else:
                result += (sym, )
        else:
            process = True
    return result

In [6]:
def bpe(text, n, k):
    for i in range(n):
        frequencies = statistics(text)
        text = merge_most_freq(text, frequencies, k)
    return text

In [7]:
n = [3, 6, 9, 12, 16, 20]
k = [8, 15, 20, 30, 40, 50]

for i,j in zip(n, k):
    result = bpe(text, i, j)
    print('--------------------------------------------------')
    print('при n = ' + str(i), ',k = ' + str(j))
    print('Всего токенов: ' + str(len(result)))
    print('|'.join(result[:100]))
    print('Самый длинный: ' + str(sorted(result, key=lambda x: len(x), reverse=True)[0]))

--------------------------------------------------
при n = 3 ,k = 8
Всего токенов: 40539
б|о|и |у| с|о|п|о|ц|к|и|н|а |и |д|р|у|с|к|ен|и|к| |з|а|ко|н|ч|и|л|и|с|ь| о|т|ст|у|п|л|ен|и|е|м| |г|ер|м|ан|ц|е|в|.| н|е|п|р|и|я|т|ел|ь|,| п|р|и|б|л|и|з|и|в|ш|и|с|ь| с| с|е|в|ер|а |к| о|с|ов|ц|у| н|а|ч|а|л| |а|р|т|и|л|л|ер
Самый длинный: и 
--------------------------------------------------
при n = 6 ,k = 15
Всего токенов: 32566
бо|и |у| с|о|по|ц|к|ин|а |и |д|ру|ск|ени|к| |за|ко|н|ч|ил|и|с|ь| о|т|ст|у|п|л|ени|е|м |г|ер|м|ан|ц|е|в|. |не|п|ри|я|те|ль|, |п|ри|б|ли|з|и|в|ш|и|с|ь| с| с|е|в|е|ра| |к| о|с|ов|ц|у| |на|ч|ал| |ар|т|ил|л|ер|и|й|ск|у|ю| |бо|р|ь|б|у| с| |к|ре|по|ст
Самый длинный: что 
--------------------------------------------------
при n = 9 ,k = 20
Всего токенов: 28708
бо|и |у| со|по|ц|ки|на |и |д|ру|ск|ени|к |за|кон|ч|и|ли|с|ь| от|ст|уп|лени|е|м |г|ер|м|ан|ц|е|в|. |не|при|я|тель|, |при|б|ли|з|и|в|ш|ис|ь| с| с|е|в|е|ра| |к| о|с|ов|ц|у| на|ч|ал| |ар|т|ил|л|ер|и|й|ск|ую| б|ор|ь|б|у| с| |к|ре|п

Он справляется не идеально, но какие-то черты токенов, близких к морфемам, проглядываются. По кажется, что чем больше n и k, тем лучше

### 2. Обучаем токенизатор из tokenizers на текстовом корпусе

In [8]:
df = pd.read_csv('data/dataset_ok.csv')
data = df['text'].tolist()
df.head(5)

,text,label
0,"наебалово века, для долбаёбов\n",INSULT
1,вся дума в таком же положении😁\n,NORMAL
2,а в каком месте массовое столкновение? шрайбик...,NORMAL
3,"значит ли это, что контроль за вывозом крупног...",NORMAL
4,вам не нужен щеночек? очень хорошие 🐶🥰\n,NORMAL


In [9]:
df['text'].to_csv('data/data.txt', index=False)

In [10]:
subtoken = CharBPETokenizer()
subtoken.train('data/data.txt', vocab_size=3000)

subtoken.save('subtoken')
subtoken = Tokenizer.from_file('subtoken')

In [11]:
N = len(data)
K = len(subtoken.get_vocab())

X = lil_matrix((N, K))
X_idf = lil_matrix((N, K))

for i, text in enumerate(data):
    token_ids = subtoken.encode(text).ids
    for t in token_ids:
        X[i, t] += 1
        if X_idf[i, t] == 0:
            X_idf[i, t] = 1

In [12]:
idf = pd.Series(X_idf.sum(axis=0).tolist()[0])
idf = idf.apply(lambda x: np.log((1 + idf.shape[0]) / (1 + x)) + 1)

In [13]:
idf[-3:]

2997    3.683691
2998    3.733701
2999    3.678825
dtype: float64

In [14]:
X = X.multiply(lil_matrix(idf.tolist()))
X_train, X_test, y_train, y_test = train_test_split(X, df['label'], test_size=0.2, stratify=df['label'])

In [15]:
classifier = SGDClassifier(loss="log", max_iter=30)
cross_val_score(classifier, X_train, y_train, scoring="f1_micro")

/home/nastia/miniconda3/envs/python3.8/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/home/nastia/miniconda3/envs/python3.8/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/home/nastia/miniconda3/envs/python3.8/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/home/nastia/miniconda3/envs/python3.8/lib/python3.8/site-packages/sklearn/linear_model/

array([0.9284598 , 0.92160097, 0.92646293, 0.9275916 , 0.9260224 ])